<a href="https://colab.research.google.com/github/Julialunna/Artificial-Intelligence/blob/main/MLP/breast_cancer_SGD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch

In [2]:
breast_cancer = load_breast_cancer()

x=breast_cancer.data
y=breast_cancer.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)  # Padroniza os dados de treino
x_test = scaler.transform(x_test)  # Padroniza os dados de teste (usando os mesmos parâmetros do treino)

# Converter para tensores do PyTorch
x_train= torch.tensor(x_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)



In [3]:
class MLP(torch.nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    #chama superclasse
    super(MLP, self).__init__()
    self.fc1 = torch.nn.Linear(input_size, hidden_size)
    self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
    self.fc3 = torch.nn.Linear(hidden_size, hidden_size)
    self.fc4 = torch.nn.Linear(hidden_size, output_size)
    #define o comportamento da rede neural
  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = torch.relu(self.fc3(x))
    x = self.fc4(x)
    return x


In [4]:
model = MLP(input_size=x_train.size()[1], hidden_size=30, output_size=1)


#defining loss function
criterion = torch.nn.BCEWithLogitsLoss()
#defining optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.009)

epochs = 100

for epoch in range(epochs):
  model.train()
  #always remember this!!
  optimizer.zero_grad()

  outputs = model(x_train)
  loss = criterion(outputs, y_train)

  #calculating gradients
  loss.backward()
  #updating weights
  optimizer.step()
  if (epoch + 1) % 10 == 0:
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 0.1469
Epoch [20/100], Loss: 0.0716
Epoch [30/100], Loss: 0.0365
Epoch [40/100], Loss: 0.0195
Epoch [50/100], Loss: 0.0073
Epoch [60/100], Loss: 0.0021
Epoch [70/100], Loss: 0.0005
Epoch [80/100], Loss: 0.0002
Epoch [90/100], Loss: 0.0001
Epoch [100/100], Loss: 0.0001


In [5]:
model.eval()
with torch.no_grad():
    outputs = model(x_test)
    #transforming outputs into probabilities
    predictions = torch.sigmoid(outputs)
    predictions = (predictions > 0.5).float()

    correct = (predictions == y_test).sum().item()
    total = y_test.size(0)
    accuracy = correct / total

    print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 98.25%
